**Narrowing down DonorsChoose for a quick triage demo** 

In [1]:
import yaml
import pandas as pd
import psycopg2
from triage.experiments import MultiCoreExperiment
from triage import create_engine
from sqlalchemy.engine.url import URL

#### DB Conn

In [2]:
with open('database.yaml', 'r') as f:
    config = yaml.safe_load(f)
    db_url = URL(
        'postgres',
        host=config['host'],
        username=config['user'],
        database=config['db'],
        password=config['pass'],
        port=config['port'],
    )

    conn = create_engine(db_url)

Steps:
- Create a sampled down version of the donors choose data
- Create a handful of features to demonstrate the feature engineering capabilities of triage
- run a small model grid with three models (Logit, DT, RF)
- Target is to get the triage run finished in a few mins

#### Creating the Sampled down version of donors

For testing, I'm creating a new projects table that contains prjects from ~10% schools in the dataset and changing the cohort query read from the "new" projects table.

Note -- There are about 57000 different schools. We can change how we sample

In [3]:
q = """

    drop table if exists optimized.projects_sampled_temp; 
    drop table if exists optimized.donations_sampled_temp;
    drop table if exists optimized.precomputed_features_sampled;
    
    -- Sampling schools and projects
    create table optimized.projects_sampled_temp as (
        with schools as (
        select 
            distinct schoolid 
        from optimized.projects
    ),
    sampled_schools as (
        select * from schools order by random() limit 1425
    )
    select 
        *
    from sampled_schools join optimized.projects using(schoolid)
    );
    
    
    -- Fetching all donations from the sampled projects
    create table optimized.donations_sampled_temp as (
        select b.* from optimized.projects_sampled_temp a join optimized.donations b using(entity_id)
    ); 
    
    -- Fetching the precomputed features for the sampled projects
    create table optimized.precomputed_features_sampled as (
        select * from optimized. projects_sampled_temp join optimized.time_series_features using(entity_id, date_posted) 
    );    
"""

conn.execute(q)

#### Triage config

Contains six features. Two static, two time-series but precomputed, two time-series calculated during triage run

In [4]:
with open('demo_config.yaml', 'r') as f:
    triage_config = yaml.safe_load(f)
    
# TODO -- replace with an S3 bucket
project_folder = '/mnt/data/experiment_data/donors/demo/'

#### Experiment

In [5]:
experiment = MultiCoreExperiment(
    config=triage_config,
    db_engine=conn,
    n_processes=2,
    n_db_processes=2,
    project_path=project_folder,
    replace=True,
    save_predictions=False
)

2021-09-20 16:32:38 - VERBOSE Matrices and trained models will be saved in /mnt/data/experiment_data/donors/demo/
2021-09-20 16:32:38 -  NOTICE Replace flag is set to true. Matrices, models, evaluations and predictions (if exist) will be replaced
2021-09-20 16:32:38 -  NOTICE Save predictions flag is set to false. Predictions won't be stored in the predictions table. This will decrease both the running time of an experiment and also decrease the space needed in the db
2021-09-20 16:32:38 - VERBOSE Using random seed [1995] for running the experiment
2021-09-20 16:32:39 -  NOTICE bias_audit_config missing in the configuration file or unrecognized. Without protected groups, you will not audit your models for bias and fairness.
2021-09-20 16:32:39 -  NOTICE scoring.subsets missing in the configuration file or unrecognized. No subsets will be generated


In [6]:
%%time
experiment.run()

2021-09-20 16:32:43 - WARNING Section: scoring - No training_metric_groups configured. If training set evaluation metrics are desired, they must be added
2021-09-20 16:32:43 - SUCCESS Experiment validation ran to completion with no errors
2021-09-20 16:32:43 - VERBOSE Computed and stored temporal split definitions
2021-09-20 16:32:43 -    INFO Setting up cohort
2021-09-20 16:32:46 - SUCCESS Cohort setted up in the table cohort_all_entities_1c7e6ce13aaaa12abe16964d559707a9 successfully
2021-09-20 16:32:46 -    INFO Setting up labels
2021-09-20 16:32:59 - SUCCESS Labels setted up in the table labels_quickstart_label_76cc17d016ee3084aed0425d67801eb1 successfully 
2021-09-20 16:32:59 -    INFO Creating features tables (before imputation) 
2021-09-20 16:32:59 -    INFO Creating collate aggregations


/home/kasun/.pyenv/versions/triage-demo/lib/python3.7/site-packages/sqlalchemy/sql/base.py:302: SAWarning: Can't validate argument 'autoload_from'; can't locate any SQLAlchemy dialect named 'autoload'
  % (k, dialect_name)


2021-09-20 16:33:00 - VERBOSE Starting Feature aggregation
2021-09-20 16:33:00 -  NOTICE Imputed feature table project_features_aggregation_imputed did not contain rows from the entire cohort, need to rebuild features
2021-09-20 16:33:01 -  NOTICE Imputed feature table teachr_funding_aggregation_imputed did not contain rows from the entire cohort, need to rebuild features
2021-09-20 16:33:01 -  NOTICE Imputed feature table donation_features_aggregation_imputed did not contain rows from the entire cohort, need to rebuild features
2021-09-20 16:33:01 -    INFO Processing query tasks with 2 processes
2021-09-20 16:33:01 -    INFO Processing features for project_features_entity_id
2021-09-20 16:33:01 -    INFO Beginning insert batch
2021-09-20 16:33:01 -    INFO Beginning insert batch
2021-09-20 16:33:02 -    INFO Beginning insert batch
2021-09-20 16:33:02 -    INFO Beginning insert batch
2021-09-20 16:33:02 -    INFO Beginning insert batch
2021-09-20 16:33:02 -    INFO Beginning insert ba

#### Model evaluations

In [9]:
q = "select run_hash from triage_metadata.triage_runs order by start_time desc limit 1;"

experiment_hash = pd.read_sql(q, conn)['run_hash'].iloc[0]
experiment_hash

'8910a7a791c557bf549f4445916eeea7'

In [10]:
q = """
    select 
        to_char(max(train_end_time), 'YYYY-MM-DD') as last_time
    from triage_metadata.experiment_models
    join triage_metadata.models using(model_hash)
    where experiment_hash = '{experiment_hash}'
""".format(experiment_hash=experiment_hash)

last_train_end_time = pd.read_sql(q, conn)['last_time'].iloc[0]
last_train_end_time

'2012-11-01'

In [11]:
q = """
    select 
        model_id, model_type, metric, parameter, best_value, worst_value, stochastic_value
    from triage_metadata.experiment_models
    join triage_metadata.models using(model_hash)
    join test_results.evaluations using(model_id)
    where experiment_hash = '{experiment_hash}' and train_end_time='{last_split}' 
    and metric='precision@' and parameter='15_pct'
""".format(
    experiment_hash = experiment_hash,
    last_split=last_train_end_time
)

evals = pd.read_sql(q, conn)
evals

,model_id,model_type,metric,parameter,best_value,worst_value,stochastic_value
0,73,sklearn.tree.DecisionTreeClassifier,precision@,15_pct,0.677419,0.161290,0.439785
1,74,triage.component.catwalk.estimators.classifier...,precision@,15_pct,0.440860,0.279570,0.378495
2,75,triage.component.catwalk.baselines.rankers.Per...,precision@,15_pct,0.451613,0.451613,0.451613
3,78,sklearn.ensemble.RandomForestClassifier,precision@,15_pct,0.419355,0.419355,0.419355
